<a href="https://colab.research.google.com/github/maiapiechocki/FER2013/blob/main/FER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [72]:
import numpy as np
import pandas as pd
import os
import cv2
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, ReLU, BatchNormalization, MaxPooling2D, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator # imagedategenerator does augmenting + preprocess data
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report


In [73]:
!nvidia-smi

Wed Jan 22 08:00:57 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   75C    P0              32W /  70W |    677MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [74]:
!pip install kaggle

!kaggle datasets download -d msambare/fer2013


Dataset URL: https://www.kaggle.com/datasets/msambare/fer2013
License(s): DbCL-1.0
fer2013.zip: Skipping, found more recently modified local copy (use --force to force download)


In [75]:
!unzip fer2013.zip

Archive:  fer2013.zip
replace test/angry/PrivateTest_10131363.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace test/angry/PrivateTest_10304478.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace test/angry/PrivateTest_1054527.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace test/angry/PrivateTest_10590091.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace test/angry/PrivateTest_1109992.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace test/angry/PrivateTest_11296953.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [87]:
!ls

best_model.keras  fer2013.zip  sample_data  test  train


In [90]:
!rm best_model.keras

rm: cannot remove 'best_model.keras': No such file or directory


In [92]:
train_path = "train"
test_path = "test"

In [93]:
print(os.listdir(train_path))
print(os.listdir(test_path))

['surprise', 'sad', 'neutral', 'angry', 'disgust', 'happy', 'fear']
['surprise', 'sad', 'neutral', 'angry', 'disgust', 'happy', 'fear']


In [94]:
# data augmentation

# define instance of imagedatagenerator for training with augmentation

train_datagen = ImageDataGenerator(
    rescale=1./255, # normalize image pixels
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    zoom_range=0.2,
    rotation_range=20,
    validation_split=0.2
)

# load and augment training data
train_generator = train_datagen.flow_from_directory(
    directory = train_path,
    target_size = (48, 48),
    batch_size = 64,
    shuffle = True,
    color_mode = "grayscale",
    class_mode = "categorical",
    subset = "training"
)

test_datagen = ImageDataGenerator(rescale=1./255, horizontal_flip=True)

test_generator = test_datagen.flow_from_directory(
    directory = test_path,
    target_size = (48,48),
    batch_size = 64,
    shuffle = True,
    class_mode = 'categorical',
    color_mode = 'grayscale'
    )

val_datagen = ImageDataGenerator(rescale=1./255,
    validation_split = 0.2)

val_generator = val_datagen.flow_from_directory(
    directory = train_path,
    target_size = (48, 48),
    batch_size = 64,
    color_mode = "grayscale",
    class_mode = "categorical",
    subset = "validation"
)

Found 22968 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.
Found 5741 images belonging to 7 classes.


In [95]:
from tensorflow.keras.regularizers import l2

def create_model():
    weight_decay = 1e-4
    model = tf.keras.models.Sequential()


    model.add(Conv2D(64, (4, 4), padding='same', kernel_regularizer=l2(weight_decay), input_shape=(48, 48, 1)))
    model.add(ReLU())
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(64, (4, 4), padding='same', kernel_regularizer=l2(weight_decay)))
    model.add(ReLU())
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(128, (4, 4), padding='same', kernel_regularizer=l2(weight_decay)))
    model.add(ReLU())
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.3))

    model.add(Conv2D(128, (4, 4), padding='same', kernel_regularizer=l2(weight_decay)))
    model.add(ReLU())
    model.add(BatchNormalization())
    model.add(Conv2D(128, (4, 4), padding='same', kernel_regularizer=l2(weight_decay)))
    model.add(ReLU())
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.4))

    model.add(tf.keras.layers.GlobalAveragePooling2D())
    # add GAP to reduce number of parameters  by reducing
    # each feature map to a single averaged value

    # fully connected layer
    model.add(tf.keras.layers.Dense(128, activation='linear'))  # linear dense layer
    model.add(ReLU())

    # output layer
    model.add(tf.keras.layers.Dense(7, activation='softmax'))  # softmax for multi-class classification

    return model

model = create_model()


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [96]:
# learning rate scheduler
initial_learning_rate = 0.001
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=initial_learning_rate,
    decay_steps=100000,  # number of steps b4 the learning rate is decayed
    decay_rate=0.9,
    staircase=True       # the decay should be applied in discrete steps (if True)
)

optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)


In [97]:
model.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy',  # since multi-class classification problem
    metrics=['accuracy']
)

In [98]:
# Callbacks

# earlystopping to stop training if validation loss does not improve
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    'best_model.keras',
    monitor='val_loss',
    save_best_only=True,
    mode='min',
    verbose=1
)

# reduceLR to reduce learning rate if validation accuracy stops improving
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=5,
    min_lr=1e-7,
    verbose=1
)


In [99]:
steps_per_epoch = train_generator.samples // train_generator.batch_size
validation_steps = validation_generator.samples // validation_generator.batch_size


In [100]:
# train the model with callbacks and steps per epoch
history = model.fit(
    train_generator,
    epochs=50,
    validation_data=validation_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    callbacks=[early_stopping, model_checkpoint, reduce_lr]
)


Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


358/358 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - accuracy: 0.2353 - loss: 1.9157
Epoch 1: val_loss improved from inf to 1.89761, saving model to best_model.keras
358/358 ━━━━━━━━━━━━━━━━━━━━ 35s 71ms/step - accuracy: 0.2354 - loss: 1.9155 - val_accuracy: 0.1543 - val_loss: 1.8976 - learning_rate: 0.0010
Epoch 2/50
  1/358 ━━━━━━━━━━━━━━━━━━━━ 8s 23ms/step - accuracy: 0.2656 - loss: 1.7811

/usr/lib/python3.11/contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)



Epoch 2: val_loss did not improve from 1.89761
358/358 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.2656 - loss: 1.7811 - val_accuracy: 0.1556 - val_loss: 1.9594 - learning_rate: 0.0010
Epoch 3/50
358/358 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 0.3151 - loss: 1.7559
Epoch 3: val_loss improved from 1.89761 to 1.67402, saving model to best_model.keras
358/358 ━━━━━━━━━━━━━━━━━━━━ 20s 54ms/step - accuracy: 0.3151 - loss: 1.7558 - val_accuracy: 0.3464 - val_loss: 1.6740 - learning_rate: 0.0010
Epoch 4/50
  1/358 ━━━━━━━━━━━━━━━━━━━━ 6s 17ms/step - accuracy: 0.3906 - loss: 1.6671
Epoch 4: val_loss improved from 1.67402 to 1.62158, saving model to best_model.keras
358/358 ━━━━━━━━━━━━━━━━━━━━ 0s 375us/step - accuracy: 0.3906 - loss: 1.6671 - val_accuracy: 0.5333 - val_loss: 1.6216 - learning_rate: 0.0010
Epoch 5/50
358/358 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - accuracy: 0.3753 - loss: 1.6328
Epoch 5: val_loss did not improve from 1.62158
358/358 ━━━━━━━━━━━━━━━━━━━━ 21s 56ms/step - ac

TypeError: This optimizer was created with a `LearningRateSchedule` object as its `learning_rate` constructor argument, hence its learning rate is not settable. If you need the learning rate to be settable, you should instantiate the optimizer with a float `learning_rate` argument.

In [106]:
from sklearn.metrics import recall_score, f1_score

evaluation = model.evaluate(validation_generator)
print(f"Model Loss: {evaluation[0]:.4f}")
print(f"Model Accuracy: {evaluation[1] * 100:.2f}%")


y_true = validation_generator.classes
y_pred = model.predict(validation_generator)
y_pred_classes = tf.argmax(y_pred, axis=-1)

y_pred_classes = y_pred_classes.numpy()  # convert from tf tensor to np array

recall = recall_score(y_true, y_pred_classes, average='macro')
f1 = f1_score(y_true, y_pred_classes, average='macro')

print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")


90/90 ━━━━━━━━━━━━━━━━━━━━ 3s 28ms/step - accuracy: 0.5477 - loss: 1.3214
Model Loss: 1.3248
Model Accuracy: 54.61%
90/90 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step
Recall: 0.1415
F1-Score: 0.1352
